# GRU Using Pytorch

In [34]:
import torch
import torch.nn as nn

# GRU Model Definition
class GRUStockModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRUStockModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])  # Take last time step output
        return out

You may need to update parent_folder and colab_base as you see fit

Download Ollama first to access llama3.2

In [23]:
# Global paths for both local (Mac) and Google Colab
PARENT_FOLDER = "/Users/colbywang/Google Drive/我的云端硬盘/Advanced NLP/Assignments/data files/organized/"
COLAB_BASE = "/content/gdrive/MyDrive/Assignments/Advanced NLP/Assignments/data files/organized/"

# RAG Function

In [25]:
import numpy as np
from llama_index.core import VectorStoreIndex, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core import SimpleDirectoryReader

# ✅ Initialize Embedding Model (Same as Before)
embedding_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
Settings.embed_model = embedding_model  # Apply globally

# ✅ Initialize Sentence Window Parser (Used for Retrieval)
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=1,
    window_metadata_key="window",
    original_text_metadata_key="original_text"
)

def build_vector_index(file_path):
    """
    Build a vector index from a given document.

    Args:
        file_path (str): Path to the document.

    Returns:
        VectorStoreIndex: Built vector store index.
    """
    # ✅ Load document
    docs = SimpleDirectoryReader(input_files=[file_path]).load_data()

    # ✅ Process nodes from document
    nodes = node_parser.get_nodes_from_documents(docs)

    # ✅ Create Vector Store Index
    index = VectorStoreIndex(nodes)

    return index

def retrieve_avg_embedding(query_text, vector_index, top_k=3):
    """
    Retrieve top-K relevant passages and compute their average embedding.

    Args:
        query_text (str): The query to search in the vector database.
        vector_index (VectorStoreIndex): Prebuilt vector store index.
        top_k (int): Number of retrieved passages to consider.

    Returns:
        np.ndarray: The averaged embedding of retrieved contexts.
    """
    # Retrieve top-K most similar documents
    retriever = vector_index.as_retriever(similarity_top_k=top_k)
    retrieved_docs = retriever.retrieve(query_text)

    # Extract embeddings of retrieved docs
    retrieved_embeddings = np.array([
        doc.embedding for doc in retrieved_docs if doc.embedding is not None
    ])

    # Handle case where no embeddings were retrieved
    if retrieved_embeddings.size == 0:
        print("⚠️ No valid embeddings found. Returning zero vector.")
        return np.zeros((embedding_model.get_embedding_dimension(),))

    # Compute the average embedding
    avg_embedding = np.mean(retrieved_embeddings, axis=0)

    return avg_embedding # shape: (384, )

# ✅ Example Usage
# file_path = "path/to/your/10-K_or_DEF14A_file.txt"
# query_text = "Summarize key financial risks."

# # Build index
# vector_index = build_vector_index(file_path)

# # Retrieve average embedding
# avg_embedding = retrieve_avg_embedding(query_text, vector_index)
# print("Avg Embedding Shape:", avg_embedding.shape)


In [ ]:
def switch_file_path(colab_path):
    """
    Converts a file path from Google Drive (Colab) to a local path.

    Args:
        colab_path (str): The file path from Google Drive in Colab.

    Returns:
        str: The equivalent local path.
    """
    local_path = colab_path.replace(COLAB_BASE, PARENT_FOLDER, 1)
    return local_path

# Train

In [44]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from glob import glob
from sklearn.model_selection import train_test_split
import random
from tqdm import tqdm

# Path to stock data folder
PARENT_FOLDER = "/Users/colbywang/Google Drive/我的云端硬盘/Advanced NLP/Assignments/data files/organized/stock-data"
all_files = glob(os.path.join(PARENT_FOLDER, "*.csv"))

# Hyperparameters
sequence_length = 7  # Lookback window
N = 1  # Predict trend in next N days
batch_size = 32
epochs = 20
hidden_size = 128
num_layers = 2
learning_rate = 0.001
threshold = 0.02  # 2% change threshold for "Stable"
train_ratio = 0.7  # 70% of files for training, 30% for testing

# Split stock files into training and testing sets
train_files, test_files = train_test_split(all_files, test_size=1-train_ratio, random_state=42)
print(f"Training on {len(train_files)} stock files, Testing on {len(test_files)} stock files")

def create_labels(df):
    """
    Create classification labels for stock price movement.
    
    Args:
        df (DataFrame): Stock data
        N (int): Number of days in the future to compute price movement
        threshold (float): Percentage change threshold to classify Up or Down

    Returns:
        Series: Labels (0 = Stable, 1 = Up, 2 = Down)
    """
    df['pct change'] = df['Close'].pct_change(N)
    df['Close_diff_pct'] = df['pct change'].shift(-N)

    # Default to "Stable"
    df['Label'] = 0  

    # Up (if change > threshold)
    df.loc[df['Close_diff_pct'] > threshold, 'Label'] = 1  

    # Down (if change < -threshold)
    df.loc[df['Close_diff_pct'] < -threshold, 'Label'] = 2  

    return df['Label']

def create_sequences(df, labels, sequence_length, N=5):
    """
    Create sequences for GRU training.

    Args:
        df (DataFrame): Stock features
        labels (Series): Classification labels
        sequence_length (int): Length of input sequence
        N (int): Lookahead period for labels

    Returns:
        np.ndarray: X (features)
        np.ndarray: y (labels)
    """
    df = df[['Close', 'High', 'Low', 'Open', 'Volume', 'CPI', 'Inflation']]
    X, y = [], []

    # Ensure labels align with future price movement
    for i in range(len(df) - sequence_length - N):
        X.append(df.values[i:i + sequence_length])  # Sequence of input features
        y.append(labels.iloc[i + sequence_length])  # Label for the next movement

    return np.array(X), np.array(y)

# Load and preprocess all training stock data
all_X_train, all_y_train = [], []

for file in tqdm(train_files, desc="Loading training files"):
    # If file is empty skip
    if os.stat(file).st_size == 0:
        continue
    
    df = pd.read_csv(file)
    if not {'Close', 'High', 'Low', 'Open', 'Volume', 'CPI', 'Inflation'}.issubset(df.columns):
        continue  # Skip if missing columns

    df = df[['Close', 'High', 'Low', 'Open', 'Volume', 'CPI', 'Inflation']]

    labels = create_labels(df)
    X, y = create_sequences(df, labels, sequence_length)

    all_X_train.append(X)
    all_y_train.append(y)

# Concatenate all training sequences
X_train = np.concatenate(all_X_train, axis=0)
y_train = np.concatenate(all_y_train, axis=0)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

# Create PyTorch DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize Model
input_size = 7  # ['Close', 'High', 'Low', 'Open', 'Volume', 'CPI', 'Inflation']
output_size = 3  # 3 classes: Up, Down, Stable
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GRUStockModel(input_size, hidden_size, num_layers, output_size).to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Model
train_losses = []
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    model.train()
    total_loss = 0
    for batch_X, batch_y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        # If batch_X has nan skip
        if torch.isnan(batch_X).any():
            continue
        
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

# Save Training Loss Plot
plt.figure(figsize=(8, 5))
plt.plot(train_losses, label="Training Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.legend()
plt.savefig("train_loss.png")
plt.close()

print("\n✅ Training complete. Training loss visualization saved.")


Training on 347 stock files, Testing on 150 stock files


Loading training files: 100%|██████████| 347/347 [02:26<00:00,  2.37it/s]


Epoch 1/20


Epoch 1/20: 100%|██████████| 60831/60831 [06:49<00:00, 148.44it/s]


Epoch 1/20, Loss: 0.6834
Epoch 2/20


...

Epoch 18/20, Loss: 0.6833
Epoch 19/20


Epoch 19/20: 100%|██████████| 60831/60831 [06:55<00:00, 146.58it/s]


Epoch 19/20, Loss: 0.6833
Epoch 20/20


Epoch 20/20: 100%|██████████| 60831/60831 [06:49<00:00, 148.51it/s]


Epoch 20/20, Loss: 0.6833

✅ Training complete. Training loss visualization saved.


In [46]:
# Save Model
torch.save(model.state_dict(), "stock_gru_model.pth")
print("✅ Model saved as 'stock_model.pth'.")

✅ Model saved as 'stock_model.pth'.


# Test

In [47]:
import os
import torch
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# ✅ Define Parent Folder for Stock Data
PARENT_FOLDER = "/Users/colbywang/Google Drive/我的云端硬盘/Advanced NLP/Assignments/data files/organized/stock-data/"

# ✅ Get list of test files
csv_files = [os.path.join(PARENT_FOLDER, file) for file in os.listdir(PARENT_FOLDER) if file.endswith(".csv")]
test_files = csv_files[int(0.7 * len(csv_files)):]

# ✅ Load GRU Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GRUStockModel(input_size=7, hidden_size=64, num_layers=2, output_size=3).to(device)
model.load_state_dict(torch.load("stock_gru_model.pth"))
model.eval()  # Set model to evaluation mode

# ✅ Initialize Metrics Storage
all_actuals, all_predictions = [], []

# ✅ Process Each Test Stock
for test_stock in tqdm(test_files, desc="stock files"):
    # If file is empty skip
    if os.stat(test_stock).st_size == 0:
        continue

    df_test = pd.read_csv(test_stock)
    if not {'Close', 'High', 'Low', 'Open', 'Volume', 'CPI', 'Inflation'}.issubset(df.columns):
        continue  # Skip if missing columns

    # Select Relevant Features
    df_test = df_test[['Close', 'High', 'Low', 'Open', 'Volume', 'CPI', 'Inflation']]

    # Generate Labels
    labels_test = create_labels(df_test)

    # Create Sequences
    X_test, y_test = create_sequences(df_test, labels_test, sequence_length=7)
    X_test, y_test = torch.tensor(X_test, dtype=torch.float32).to(device), torch.tensor(y_test, dtype=torch.long).to(device)

    # ✅ Run Inference
    with torch.no_grad():
        for i in range(len(X_test)):
            X_sample = X_test[i].unsqueeze(0)  # Add batch dimension

            if torch.isnan(X_sample).any():
                continue

            y_actual = y_test[i].item()
            output = model(X_sample)
            _, predicted = torch.max(output, 1)
            
            # Store results
            all_actuals.append(y_actual)
            all_predictions.append(predicted.item())

# ✅ Compute Classification Metrics
accuracy = accuracy_score(all_actuals, all_predictions)
precision, recall, f1, _ = precision_recall_fscore_support(all_actuals, all_predictions, average="weighted", zero_division=0)

# ✅ Print Results
print("\n🔹 **Overall Test Set Performance Metrics** 🔹")
print(f"✅ Accuracy: {accuracy:.4f}")
print(f"📏 Precision: {precision:.4f}")
print(f"📡 Recall: {recall:.4f}")
print(f"⚖️ F1-Score: {f1:.4f}")


stock files: 100%|██████████| 150/150 [06:29<00:00,  2.60s/it]



🔹 **Overall Test Set Performance Metrics** 🔹
✅ Accuracy: 0.7716
📏 Precision: 0.6080
📡 Recall: 0.7716
⚖️ F1-Score: 0.6726
